<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Run GriddedCCP

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [5]:
datasets_32 = ["mnist","celeba32",  "svhn"]
datasets_64 = ["celeba64",  "zs-multi-mnist"]

Grided_ccp_kwargs = [("GridedCCP", ""),
                   #("GridedCCP_large", "--n-blocks 7 --kernel-size 9 --init-kernel-size 11 --n-conv-layers 2"),
                    ]

args_32_ccp = [parse_arguments("GridedCCP {} --starting-run {} --name {} --chckpnt-dirname results/iclr_bug/ {}".format(d, s, name, other).split())
                for s in range(2)
                for d in datasets_32
                for name,other in Grided_ccp_kwargs]

args_64_ccp = [parse_arguments("GridedCCP {} --starting-run {} --name {} --chckpnt-dirname results/iclr_bug/ {}".format(d, s, name, other).split())
            for s in range(3)
            for d in datasets_64
            for name,other in Grided_ccp_kwargs]

In [6]:
executor_32_ccp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_32_ccp.update_parameters(num_gpus=1, 
                                   time=60*24*2,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                   ntasks_per_node=3
                                  )

executor_64_ccp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_64_ccp.update_parameters(num_gpus=1, 
                                   time=60*24*2,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                   ntasks_per_node=2
                                  )

In [7]:
job_32_ccp = executor_32_ccp.map_array(Run(), args_32_ccp)
job_64_ccp = executor_64_ccp.map_array(Run(), args_64_ccp)

In [16]:
############################################################

In [9]:
job_32_ccp

[SlurmJob<job_id=25650155_0, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650155_1, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650155_2, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650155_3, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650155_4, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650155_5, task_id=None, state="RUNNING">]

In [8]:
job_64_ccp

[SlurmJob<job_id=25650156_0, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650156_1, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650156_2, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650156_3, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650156_4, task_id=None, state="RUNNING">,
 SlurmJob<job_id=25650156_5, task_id=None, state="RUNNING">]

In [15]:
for j in job_32_ccp:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2020-04-22 08:48:44,270) - Starting with JobEnvironment(job_id=25526133_0, hostname=learnfair0266, local_rank=0(3), node=0(1), global_rank=0(3))
submitit INFO (2020-04-22 08:48:44,270) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/25526133_0/25526133_0_submitted.pkl

--- Training mnist/GridedCCP/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1       -0.8811       -1.0171     +  160.4763
      2       -1.0978       -1.1700     +  158.5720
      3       -1.1815       -1.1940     +  158.6497
      4       -1.2043       -1.2099     +  158.6845
      5       -1.2087       -1.2105     +  158.6931
      6       -1.2180       -1.2099        159.4040
      7       -1.2220       -1.2215     +  156.0196
      8       -1.2344       -1.2319     +  155.2738
      9       -1.2308       -1.2385     +  155.5703
     10       -1.2353       -1.2431     +  1

In [16]:
for j in job_32_ccp:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit